#Nhóm X


In [328]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import threading
import nltk
from threading import Thread, current_thread
from multiprocessing import Process, current_process
from nltk import tokenize
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
from sklearn import preprocessing
from sklearn import model_selection
from lxml import etree



In [283]:
# first page ------------------
URL = 'https://batdongsan.vn/ban-nha/'
url_check = 'https://batdongsan.vn/ban-nha/p100000asdsad00'
list_url = []

# element page -----------------
# parse  string list[dict{str: str}]
dict_html_parsing = [{'body':''},\
                    {'div': 'landtech-container'},\
                    {'div':  "uk-container uk-container-center"},\
                    {'div': "block-custom realestate-post-detail-custom"},\
                    {'div': "uk-grid uk-grid-medium"},\
                    {'div': "uk-width-medium-3-5"},\
                    {'div': "project-global-object-block-003 information-custom"},\
                    {'div': "wrapper"},\
                    {'div': "uk-panel"}]

dict_html_house_attribs = list(x for x in dict_html_parsing)
dict_html_house_title = list(x for x in dict_html_parsing)
dict_html_house_price = list(x for x in dict_html_parsing)

dict_html_house_title.extend([{'h1': "uk-panel-title"},\
                                                {'span':''}])

dict_html_house_price.extend([{'div': "item"},\
                                                {'div': "body"},\
                                                {'div': "meta"},\
                                                {'strong': "price"}])

dict_html_house_attribs.extend([{'div': "item"},\
                                                {'div': "body"},\
                                                {'div': "param"},\
                                                {'ul': "uk-list"}])


In [163]:
header_features = ['Title', 'Price', 'Address', 'Area', 'Room', 'WC' , 'Description']
house_dataframe = pd.DataFrame(columns = header_features)
print(house_dataframe)


Empty DataFrame
Columns: [Title, Price, Address, Area, Room, WC, Description]
Index: []


# Data Collection 


In [42]:
class Address:
    def __init__(self, city, district, ward, street):
        self._city = city
        self._district = district
        self._ward = ward
        self._street = street
    
    def display(self):
        print(self._street, self._ward, self._district, self._city)

In [257]:
class House:
    def __init__(self, title,  price ,address, area, room, wc, description):
        self._title = title
        self._price = price
        self._address = address
        self._area= area 
        self._room = room 
        self._wc = wc
        self._description = description
    
    def scrappingDataWeb(self, url):
        page = requests.get(url)
        
        # title parsing
        soup_title = BeautifulSoup(page.content, "html.parser")
        for item in dict_html_house_title:
            if list(item.values())[0] != '':
                temp = soup_title.find(list(item.keys())[0], class_ = item.get(list(item.keys())[0]))
                soup_title = temp
            else:
                temp = soup_title.find(list(item.keys())[0])
                soup_title = temp
        self._title = soup_title.text
        
        # price parsing
        soup_price = BeautifulSoup(page.content, "html.parser")
        for item in dict_html_house_price:
            if item.get(list(item.keys())[0]) != '':
                soup_price = soup_price.find(list(item.keys())[0], class_ = item.get(list(item.keys())[0]))
            else:
                soup_price = soup_price.find(list(item.keys())[0])
        self._price = soup_price.text
        
        ## parsing another attributes
        soup_attrib = BeautifulSoup(page.content, "html.parser")
        for item in dict_html_house_attribs:
            if list(item.values())[0] != '':
                soup_attrib = soup_attrib.find(list(item.keys())[0], class_ = item.get(list(item.keys())[0]))
            else: 
                soup_attrib = soup_attrib.find(list(item.keys())[0])
        attrib_root = soup_attrib.find_all("li")
        for root in attrib_root:
            if root.find('strong').text == "Diện tích:":
                self._area = root.text
            if root.find('strong').text == "Địa chỉ:":
                self._address == root.text
            if root.find('strong').text == "Phòng ngủ:":
                self._room = root.text
        
        soup_des = BeautifulSoup(page.content, "html.parser")
        dom = etree.HTML(str(soup_des))
        xpath_str = '//*[@id="post-detail"]/body/div[7]/div/div/div[1]/div[1]/div/div/div/div/div/p' # The XPath expression for the blog's title
        soup_des = (dom.xpath(xpath_str)[0])
        temp_text = soup_des.text
        for item in soup_des.iter():
            temp_text = ''.join([temp_text, item.tail])
        self._description = temp_text
        ## chỉnh sửa phần này thành nhiều thành phần attib
        
    
    ## attributes matching for finding address base on The compare with voacabulary embedded database with Levhein distance
    def tranferDescription(self, embedded_database): 
        temp_address = Address('','','','')
        for city in embedded_database:
            for district in city._container:
                ## tokenize description scrapping from web to list vocab
                for token in tokenize(self._description):
                    # compare each vocab with each item address from db
                    score = fuzz(token, district)
                    if score >= 0.7:
                        Address._district = district
        return
    
    ##
    def tranferAddress(self):
        return
    def display(self):
        print(self._title, "\n",\
              self._price, "\n",\
              self._address, "\n",\
              self._area, "\n",\
              self._room, "\n",\
              self._wc, "\n",\
              self._description)

In [ ]:
temp = House("","","","","","","")
temp.scrappingDataWeb("https://batdongsan.vn/ban-nha-pho-cu-loc-thanh-xuan-56m-4-tang-gia-545-ty-r268420")
temp.display()

In [337]:
def process_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return True
    else:
        return False

class HouseContainer:
    def __init__(self, list_in = []):
        self._house_container = list_in
        self._url_container = []
        self._url_house_container = []
        self._size = len(self._house_container)
        
    
    def parsingUrlContainer(self):
        index = 1
        temp = URL
        while check(temp) == True:
            self._url_container.append(temp)
            index +=1;
            elem = ''.join(['p',str(index)])
            temp = ''.join([URL, elem])
            print(temp)
        
    def pasingHouseContainer(self, url):
        return
        
    def containerToDataframe(self):
        return
    
    def displayUrl(self):
        for item in self._url_container:
            print(item)
    
    def displayHouse(self):
        for house in self._house_container:
            house.display()
    

In [338]:
temp= HouseContainer([])
temp.parsingUrlContainer()
temp.displayUrl()


AttributeError: 'Process' object has no attribute 'status_code'